In [1]:
import os

import numpy as np
import vtk
from vtk.util.numpy_support import numpy_to_vtk, vtk_to_numpy

import pyfocusr
from pyfocusr.vtk_functions import read_vtk_mesh

In [2]:
location_meshes = '../data/'
target_filename = 'target_mesh.vtk'
source_filename = 'source_mesh.vtk'

target_vtk_mesh = read_vtk_mesh(os.path.join(location_meshes, target_filename))
source_vtk_mesh = read_vtk_mesh(os.path.join(location_meshes, source_filename))

In [3]:
reg = pyfocusr.Focusr(vtk_mesh_target=target_vtk_mesh, 
                      vtk_mesh_source=source_vtk_mesh,
                      get_weighted_spectral_coords=False,
                      list_features_to_calc=['curvature'],
                      rigid_reg_max_iterations=20,
                      non_rigid_alpha=0.5,
                      non_rigid_beta=3,
                      non_rigid_max_iterations=20)

### View mesh features

In [5]:
reg.graph_source.view_mesh_features(feature_idx=1)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

### View un-ordered/corrected/weighted eigenvectors (spectral coordinates)

#### Source Eigenvector 1

In [10]:
reg.graph_source.view_mesh_eig_vec(eig_vec=2)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

#### Target Eigenvector 1

In [11]:
reg.graph_target.view_mesh_eig_vec(eig_vec=2)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

# Notes about comparison: 
Depending on if we include features ('curvature') or not, we will get different results for these meshes. 

If we leave the list at `list_features_to_calc` as an empty list `[]` then: <br>
We note that the color gradient is the same (in the same direction) between the two meshes. However, the colours are flipped. This is because the direction of "positive" can be flipped from one mesh to another. This will (should) be corrected automatically before the registration begins. 

If we include `'curvature'` in `list_features_to_calc` with `['curvature']` then: <br>
We get eigenvectors that align in direction and orientation (at least for the first 3 (0, 1, 2). 

Note, the flipped nature of the spectral coordinates can/will be corrected in the algorithm (if it exists). However, this only works because these are both of the same leg (right - as evident by the large side of the trochlear groove) and will likely faily/break if a left and right leg were analyzed. Therefore, all legs should be flipped to be the same side to "fix" things. 

## Begin Registration
This will print out the registration parameters/results as it goes. <br>
- First it will print out the "pairs" of eigenvectors between the two meshes and which ones on the source were flipped to match the target eigenvectors. 
- Second it will print the rigid registration progress and ultimately the registration "results"/"parameters"
- Third it will print the same for the non-rigid registration progress. 


In [12]:
reg.align_maps()

Eigenvector Sorting Results

The matches for eigenvectors were as follows:
Target	|  Source
     0	|  0     
     1	|  -1    
     2	|  2     
     3	|  -3    
     4	|  -4    
     5	|  5     
*Negative source values means those eigenvectors were flipped*


Rigid Registration Beginning

Iteration:1
ML:-50155.768; 	ML change (error): 50155.768; 	Sigma^2:     0.068; 	Sigma^2 change:     0.104
[===                                                                     ]
Iteration:2
ML:-53768.487; 	ML change (error):  3612.718; 	Sigma^2:     0.062; 	Sigma^2 change:     0.006
[=======                                                                 ]
Iteration:3
ML:-53904.790; 	ML change (error):   136.303; 	Sigma^2:     0.055; 	Sigma^2 change:     0.007
[==========                                                              ]
Iteration:4
ML:-54145.160; 	ML change (error):   240.370; 	Sigma^2:     0.046; 	Sigma^2 change:     0.009
[==============                                               

Iteration:19
ML:-66779.170; 	ML change (error):   653.063; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[====================================================================    ]
Iteration:20
ML:-67331.346; 	ML change (error):   552.177; 	Sigma^2:     0.000; 	Sigma^2 change:     0.000
[========================================================================]
Registration Performance Metrics
Time to initialize EM: 0.08394598960876465
Time to initialize registration: 13.341288328170776
Average Expectation Time:                    0.6060 +/- 0.0878
Average Maximization Time:                   0.1300 +/- 0.0193
Maximization Times - Per individual step
Average Update Transform Time:               0.1289 +/- 0.0191
Average Transform Time:                      0.0007 +/- 0.0004
Average Update Variance Time:                0.0003 +/- 0.0001

Number of iterations performed:          20
Error at time of finish:                 552.1767731662403


## View the spectral coordinates

In [14]:
reg.view_aligned_spectral_coords(include_rigid_aligned=True)

Viewer(point_set_colors=array([[0.8392157 , 0.        , 0.        ],
       [0.54901963, 0.23137255, 1.       …